In [1]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.metrics import classification_report, confusion_matrix

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
image_path = '/kaggle/input/k-fold-images/images'

In [3]:
input_shape = (299, 299) 
batch_size = 32
num_epochs = 31
num_folds = 4 

In [4]:
data_gen = ImageDataGenerator(
                    rescale=1./255,
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    zoom_range=0.2,
                    horizontal_flip=True,
                    vertical_flip=False,
                    rotation_range=10,  
                    shear_range=0.2,
                    brightness_range=[0.8, 1.2])

In [5]:
classes = os.listdir(image_path)

all_image_paths = []
all_labels = []

for class_name in classes:
    class_path = os.path.join(image_path, class_name)
    class_images = os.listdir(class_path)
    class_image_paths = [os.path.join(class_path, image) for image in class_images]
    all_image_paths.extend(class_image_paths)
    all_labels.extend([class_name] * len(class_image_paths))

all_image_paths = np.array(all_image_paths)
all_labels = np.array(all_labels)

stratified_kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

In [6]:
len(classes)

5

In [7]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(*input_shape, 3))
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(len(classes), activation='softmax')
])


83683744/83683744 [==============================] - 0s 0us/step


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
for fold, (train_indices, val_indices) in enumerate(stratified_kfold.split(all_image_paths, all_labels)):
    train_data = all_image_paths[train_indices]
    train_labels = all_labels[train_indices]
    val_data = all_image_paths[val_indices]
    val_labels = all_labels[val_indices]
    print("################# Results for fold:-", fold+1, "###################")
    train_generator = data_gen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filepath': train_data, 'class': train_labels}),
        x_col='filepath',
        y_col='class',
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=True,
    )

    val_generator = data_gen.flow_from_dataframe(
        dataframe=pd.DataFrame({'filepath': val_data, 'class': val_labels}),
        x_col='filepath',
        y_col='class',
        target_size=input_shape,
        batch_size=batch_size,
        class_mode='categorical',
        shuffle=False,
    )
    

    Checkpoint = ModelCheckpoint("K-Xception.hdf5", monitor="val_accuracy", save_best_only=True, mode="max")
    model.fit(
        train_generator,
        epochs=num_epochs,
        validation_data=val_generator,
        callbacks=[Checkpoint]
    )
    
    new_model=tf.keras.models.load_model("/kaggle/working/K-Xception.hdf5")
    
    predictions = new_model.predict(val_generator)
    predicted_labels = np.argmax(predictions, axis=1)
    true_labels = val_generator.classes

    class_names = list(val_generator.class_indices.keys())
    report = classification_report(true_labels, predicted_labels, target_names=class_names)
    
    print(f"Fold {fold + 1}:")
    print("Classification Report:")
    print(report)

    # Calculate and display the confusion matrix
    confusion_mat = confusion_matrix(true_labels, predicted_labels)
    print("Confusion Matrix:")
    print(confusion_mat)

model.save('xception_model.h5')


################# Results for fold:- 1 ###################
Found 8076 validated image filenames belonging to 5 classes.
Found 2019 validated image filenames belonging to 5 classes.
Epoch 1/31
253/253 [==============================] - 426s 2s/step - loss: 0.5307 - accuracy: 0.7995 - val_loss: 0.7086 - val_accuracy: 0.7984
Epoch 2/31
253/253 [==============================] - 305s 1s/step - loss: 0.3813 - accuracy: 0.8519 - val_loss: 1.8229 - val_accuracy: 0.6632
Epoch 3/31
253/253 [==============================] - 305s 1s/step - loss: 0.3303 - accuracy: 0.8717 - val_loss: 0.4100 - val_accuracy: 0.8489
Epoch 4/31
253/253 [==============================] - 296s 1s/step - loss: 0.2885 - accuracy: 0.8836 - val_loss: 0.4910 - val_accuracy: 0.8336
Epoch 5/31
253/253 [==============================] - 297s 1s/step - loss: 0.3023 - accuracy: 0.8817 - val_loss: 0.3788 - val_accuracy: 0.8534
Epoch 6/31
253/253 [==============================] - 296s 1s/step - loss: 0.2808 - accuracy: 0.8895 - v